In [1]:
import socket
import logging
import threading
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from os import urandom
import selectors
from cryptography.fernet import Fernet

# Упражнение 1

1. Расширение функционала клиента.

* Модифицируйте код клиентского приложения так, чтобы он мог читать сообщения, введенные пользователем в консоли, и отправлять их серверу. После отправки каждого сообщения клиент должен получать и выводить эхо-ответ от сервера.

* Реализуйте возможность для пользователя завершить работу клиентского приложения по специальной команде, например, введя `exit`. Это должно корректно закрывать соединение с сервером и останавливать программу клиента.

2. Обработка исключений.

* Добавьте в код сервера обработку исключений для ситуаций, когда клиент неожиданно разрывает соединение. Убедитесь, что сервер продолжает работу и готов принимать новые подключения даже после возникновения ошибок.

* Реализуйте обработку исключений в клиентском приложении для обработки ошибок сети, таких как невозможность подключиться к серверу. В случае ошибки должно выводиться соответствующее сообщение, и клиент должен корректно завершать работу.

3. Расширение функционала сервера.

* Дополните сервер функционалом логирования. Сервер должен записывать в лог-файл информацию о каждом подключении: время подключения и адрес клиента. Также сервер должен логировать сообщения, получаемые от клиентов.

* Разработайте механизм обработки специальных команд от клиента, например, `shutdown`, который будет корректно завершать работу сервера. Убедитесь, что сервер закрывает все активные соединения перед выключением

In [9]:
def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 9090))
    server_socket.listen()
    print("Сервер запущен и ожидает подключений...")
    client_socket, client_address = server_socket.accept()
    print("Соединение установлено =D\n")
    
    logging.basicConfig(
    level=logging.INFO, 
    filename = "mylog.log", 
    format = "%(asctime)s - %(message)s " 
    )

    try:
        while True:
            data = client_socket.recv(1024)
            logging.info(f'client adress: {client_address} - client message: {data.decode("utf-8")}')  
            if data:
                client_socket.sendall(data)
                
                if data.decode('utf-8')=='exit':
                    client_socket.close()
                    print("Соединение с клиентом закрыто")
                    print("Сервер ожидает подключений...")
                    client_socket, client_address = server_socket.accept()
                    print("Соединение установлено =D\n")

                if data.decode('utf-8')=='shutdown':
                    break
            else:
                client_socket.close()
                print("Соединение с клиентом разорвано")
                print("Сервер ожидает подключений...")
                client_socket, client_address = server_socket.accept()
                print("Соединение установлено =D\n")

    finally:
        print("Соединение с клиентом закрыто\nСервер закрыт")
        client_socket.close()
        server_socket.close()

In [10]:
start_server()

Сервер запущен и ожидает подключений...
Соединение установлено =D

Соединение с клиентом закрыто
Сервер закрыт


# Упражнение 2
* Запустите модифицированный сервер и подключите к нему несколько клиентов одновременно (используя код клиента из Упражнения 1). Проверьте, что сервер корректно обрабатывает каждое соединение в отдельном потоке, возвращая эхо-ответы.

* Исследуйте, как Global Interpreter Lock (GIL) влияет на многопоточность в Python, и расскажите, почему многопоточность остается полезной в сетевом программировании, несмотря на присутствие GIL.

In [4]:
def handle_client(client_socket, server_socket):
    """Обрабатывает соединение с клиентом в отдельном потоке."""
    try:
        while True:
            message = client_socket.recv(1024)
            client_socket.sendall(message)
            if message.decode('utf-8') in ('exit', None):
                break  # Клиент закрыл соединение
    finally:
        client_socket.close()
        if message.decode('utf-8') == 'shutdown':
            server_socket.close()

def start_server():
    """Запускает сервер и обрабатывает множество соединений."""
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 9090))
    server_socket.listen()
    print("Сервер запущен и ожидает подключений...")

    while True:
        client_sock, address = server_socket.accept()
        print(f"Принято соединение от {address}")
        client_thread = threading.Thread(target=handle_client, args=(client_sock, server_socket))
        client_thread.start()

In [ ]:
start_server()

Сервер запущен и ожидает подключений...
Принято соединение от ('127.0.0.1', 61327)
Принято соединение от ('127.0.0.1', 61343)


Из-за GIL только один поток может исполнять Python-код в каждый момент времени, что ограничивает использование многопроцессорных систем для вычислительно-интенсивных задач.

В сетевом программировании операции ввода-вывода часто блокируют потоки. Пока один поток ждет данные, другие потоки могут продолжать работу, что эффективно использует время ожидания. Потоки могут обрабатывать асинхронные события параллельно, улучшая отзывчивость приложений. Многопоточный код часто проще для понимания и сопровождения, позволяя обрабатывать разные соединения в отдельных потоках.

# Упражнение 3. Реализация протокола Диффи-Хеллмана и асимметричное шифрование

In [15]:
parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())

private_key = parameters.generate_private_key()
public_key = private_key.public_key()

In [41]:
shared_secret = private_key.exchange(public_key)
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info='handshake data'.encode(),
    backend=default_backend()
).derive(shared_secret)

In [42]:
iv = urandom(16)
cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update("Секретное сообщение".encode()) + encryptor.finalize()

In [43]:
decryptor = cipher.decryptor()
plaintext = decryptor.update(ciphertext) + decryptor.finalize()

In [44]:
plaintext.decode('utf-8')

'Секретное сообщение'

In [5]:
def create_shared_secret(client_socket, private_key, params_bytes, public_key_b):
    client_socket.sendall(params_bytes)
    client_socket.sendall(public_key_b)
    
    client_public_key_b = client_socket.recv(1024)
    client_public_key = serialization.load_pem_public_key(client_public_key_b)
    
    shared_secret = private_key.exchange(client_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info='handshake data'.encode(),
        backend=default_backend()
    ).derive(shared_secret)
    
    return derived_key


def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 9090))
    server_socket.listen()
    print("Сервер запущен и ожидает подключений...")
    client_socket, client_address = server_socket.accept()
    
    parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    
    public_key_b = public_key.public_bytes(
    encoding = serialization.Encoding.PEM,
    format = serialization.PublicFormat.SubjectPublicKeyInfo
    )
    params_bytes = parameters.parameter_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.ParameterFormat.PKCS3
    )
    derived_key = create_shared_secret(client_socket, private_key, params_bytes, public_key_b)
    
    print("Соединение установлено =D\n")
    
    logging.basicConfig(
    level=logging.INFO, 
    filename = "mylog.log", 
    format = "%(asctime)s - %(message)s " 
    )

    try:
        while True:
            iv = client_socket.recv(1024)
            ciphertext = client_socket.recv(1024)
            
            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            decryptor = cipher.decryptor()
            data = decryptor.update(ciphertext) + decryptor.finalize()

            logging.info(f'client adress: {client_address} - client message: {data.decode("utf-8")}')
            
            if data:
                iv = urandom(16)
                cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
                encryptor = cipher.encryptor()
                ciphertext = encryptor.update(data) + encryptor.finalize()

                client_socket.sendall(iv)
                client_socket.sendall(ciphertext)

                if data.decode('utf-8') == 'exit':
                    client_socket.close()
                    print("Соединение с клиентом закрыто")
                    print("Сервер ожидает подключений...")
                    client_socket, client_address = server_socket.accept()
                    derived_key = create_shared_secret(client_socket, private_key, params_bytes, public_key_b)
                    print("Соединение установлено =D\n")

                if data.decode('utf-8') == 'shutdown':
                    break
            else:
                client_socket.close()
                print("Соединение с клиентом разорвано")
                print("Сервер ожидает подключений...")
                client_socket, client_address = server_socket.accept()
                derived_key = create_shared_secret(client_socket, private_key, params_bytes, public_key_b)
                print("Соединение установлено =D\n")

    finally:
        print("Соединение с клиентом закрыто\nСервер закрыт")
        client_socket.close()
        server_socket.close()

In [6]:
start_server()

Сервер запущен и ожидает подключений...
Соединение установлено =D

Соединение с клиентом закрыто
Сервер ожидает подключений...
Соединение установлено =D

Соединение с клиентом закрыто
Сервер закрыт


# Упражнение 5. Работа с UDP-сокетами

In [18]:
def start_udp_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.bind(('localhost', 9090))
    print("Сервер запущен и ожидает сообщений...")

    while True:
        data, client_address = server_socket.recvfrom(1024)
        server_socket.sendto(data, client_address)
        
        if data.decode()=='exit':
            print("Сервер закрыт")
            server_socket.close()
            break
            
        print(f'Отправлен эхо-ответ "{data.decode()}" на адрес {client_address}')

In [20]:
start_udp_server()

Сервер запущен и ожидает сообщений...
Отправлен эхо-ответ "ghjkl" на адрес ('127.0.0.1', 51993)
Отправлен эхо-ответ "okj" на адрес ('127.0.0.1', 51993)
Отправлен эхо-ответ "vdvj" на адрес ('127.0.0.1', 51993)
Сервер закрыт


# Упражнение 6. Использование селекторов для обработки множественных соединений

In [22]:
selector = selectors.DefaultSelector()
server_running = True

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

def accept_connection(server_socket):
    try:
        client_socket, client_address = server_socket.accept()
        logger.info(f"Подключение от {client_address}")
        client_socket.setblocking(False)
        selector.register(client_socket, selectors.EVENT_READ, send_echo)
    except Exception as e:
        logger.error(f"Ошибка при подключении клиента: {e}")

def send_echo(client_socket):
    try:
        data = client_socket.recv(1024)

        if data:
            client_socket.sendall(data)

            if data.decode('utf-8') == 'shutdown':
                shutdown_server()
                return

            if data.decode('utf-8') == 'exit':
                close_connection(client_socket)
                return
        else:
            close_connection(client_socket, 1)
    except Exception as e:
        logger.error(f"Ошибка при обработке данных от клиента: {e}")
        close_connection(client_socket)

def close_connection(client_socket, break_ = 0):
    try:
        selector.unregister(client_socket)
        client_socket.close()

        if not break_:
            logger.info("Соединение с клиентом закрыто")
        else:
            logger.info("Соединение с клиентом разорвано")
            
    except Exception as e:
        logger.error(f"Ошибка при закрытии соединения: {e}")

def shutdown_server():
    global server_running
    server_running = False
    events = selector.select(timeout=None)
    for key, _ in events:
        if key.data == send_echo:
            close_connection(key.fileobj)

def start_server():
    global server_running
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 9090))
    server_socket.listen()
    server_socket.setblocking(False)
    selector.register(server_socket, selectors.EVENT_READ, accept_connection)

    logger.info("Сервер запущен и ожидает подключений...")
    while server_running:
        try:
            events = selector.select(timeout=None)
            for key, _ in events:
                callback = key.data
                callback(key.fileobj)
        except Exception as e:
            logger.error(f"Ошибка в основном цикле сервера: {e}")

    logger.info("Сервер закрыт")
    server_socket.close()

In [23]:
start_server()

2024-05-20 09:04:27,831 - Сервер запущен и ожидает подключений...
2024-05-20 09:04:35,123 - Подключение от ('127.0.0.1', 54374)
2024-05-20 09:04:38,962 - Подключение от ('127.0.0.1', 54376)
2024-05-20 09:04:50,519 - Соединение с клиентом закрыто
2024-05-20 09:04:57,149 - Соединение с клиентом закрыто
2024-05-20 09:04:57,150 - Сервер закрыт


# Лабораторная работа №8

In [16]:
server_running = True
lock = threading.Lock()

def create_shared_secret(client_socket, private_key, params_bytes, public_key_b):
    client_socket.sendall(params_bytes)
    client_socket.sendall(public_key_b)
    
    client_public_key_b = client_socket.recv(1024)
    client_public_key = serialization.load_pem_public_key(client_public_key_b)
    
    shared_secret = private_key.exchange(client_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info='handshake data'.encode(),
        backend=default_backend()
    ).derive(shared_secret)
    
    return derived_key

def handle_client(client_socket, server_socket, derived_key):
    """Обрабатывает соединение с клиентом в отдельном потоке."""
    global server_running
    try:
        while True:
            iv = client_socket.recv(1024)
            ciphertext = client_socket.recv(1024)
            
            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend = default_backend())
            decryptor = cipher.decryptor()
            data = decryptor.update(ciphertext) + decryptor.finalize()
            
            iv = urandom(16)
            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            encryptor = cipher.encryptor()
            ciphertext = encryptor.update(data) + encryptor.finalize()

            client_socket.sendall(iv)
            client_socket.sendall(ciphertext)

            if data.decode('utf-8') in ('exit', None):
                break
                
            if data.decode('utf-8') == 'shutdown':
                with lock:
                    server_running = False
                break

    finally:
        print("Соединение с клиентом закрыто")
        client_socket.close()
        if not server_running:
            with lock:
                server_socket.close()
            print("Сервер закрыт")

def start_server():
    """Запускает сервер и обрабатывает множество соединений."""
    global server_running
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 9090))
    server_socket.listen()
    
    parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    
    public_key_b = public_key.public_bytes(
    encoding = serialization.Encoding.PEM,
    format = serialization.PublicFormat.SubjectPublicKeyInfo
    )
    params_bytes = parameters.parameter_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.ParameterFormat.PKCS3
    )
    print("Сервер запущен и ожидает подключений...")
    
    server_running = True
    while server_running:
        try:
            client_socket, address = server_socket.accept()
        except socket.error:
            break
        derived_key = create_shared_secret(client_socket, private_key, params_bytes, public_key_b)
        print(f"Принято соединение от {address}")
        client_thread = threading.Thread(target=handle_client, args=(client_socket, server_socket, derived_key))
        client_thread.start()

In [17]:
start_server()

Сервер запущен и ожидает подключений...
Принято соединение от ('127.0.0.1', 56337)
Соединение с клиентом закрыто
Сервер закрыт


In [6]:
def start_udp_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.bind(('localhost', 9090))

    key = Fernet.generate_key()
    cipher = Fernet(key)
    print("Сервер запущен и ожидает сообщений...")
    
    _, client_address = server_socket.recvfrom(1024)
    server_socket.sendto(key, client_address)

    while True:
        data, client_address = server_socket.recvfrom(1024)
        data = cipher.decrypt(data)
        
        ciphertext = cipher.encrypt(data)
        server_socket.sendto(ciphertext, client_address)

        if data.decode()=='exit':
            print("Сервер закрыт")
            server_socket.close()
            break
            
        print(f'Отправлен эхо-ответ "{data.decode()}" на адрес {client_address}')

In [7]:
start_udp_server()

Сервер запущен и ожидает сообщений...
Отправлен эхо-ответ "ыдаодыоа" на адрес ('127.0.0.1', 58046)
Отправлен эхо-ответ "ыдаыдола" на адрес ('127.0.0.1', 58046)
Сервер закрыт


In [2]:
selector = selectors.DefaultSelector()
server_running = True

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

ciphers = {}

def accept_connection(server_socket):
    try:
        client_socket, client_address = server_socket.accept()
        
        key = Fernet.generate_key()
        cipher = Fernet(key)
        ciphers[client_socket] = cipher
        client_socket.sendall(key)
        
        logger.info(f"Подключение от {client_address}")
        client_socket.setblocking(False)
        selector.register(client_socket, selectors.EVENT_READ, send_echo)
    except Exception as e:
        logger.error(f"Ошибка при подключении клиента: {e}")

def send_echo(client_socket):
    try:
        cipher = ciphers[client_socket]
        
        ciphertext = client_socket.recv(1024)
        data = cipher.decrypt(ciphertext)
        if data:
            ciphertext = cipher.encrypt(data)
            client_socket.sendall(ciphertext)

            if data.decode('utf-8') == 'shutdown':
                shutdown_server()
                return

            if data.decode('utf-8') == 'exit':
                close_connection(client_socket)
                return
        else:
            close_connection(client_socket, 1)
    except Exception as e:
        logger.error(f"Ошибка при обработке данных от клиента: {e}")
        close_connection(client_socket)

def close_connection(client_socket, break_ = 0):
    try:
        ciphers.pop(client_socket)
        selector.unregister(client_socket)
        client_socket.close()

        if not break_:
            logger.info("Соединение с клиентом закрыто")
        else:
            logger.info("Соединение с клиентом разорвано")
            
    except Exception as e:
        logger.error(f"Ошибка при закрытии соединения: {e}")

def shutdown_server():
    global server_running
    server_running = False
    events = selector.select(timeout=None)
    for key, _ in events:
        if key.data == send_echo:
            close_connection(key.fileobj)

def start_server():
    global server_running
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 9090))
    server_socket.listen()
    server_socket.setblocking(False)
    selector.register(server_socket, selectors.EVENT_READ, accept_connection)

    logger.info("Сервер запущен и ожидает подключений...")
    while server_running:
        try:
            events = selector.select(timeout=None)
            for key, _ in events:
                callback = key.data
                callback(key.fileobj)
        except Exception as e:
            logger.error(f"Ошибка в основном цикле сервера: {e}")

    logger.info("Сервер закрыт")
    server_socket.close()

In [3]:
start_server()

2024-05-20 16:36:56,441 - Сервер запущен и ожидает подключений...
2024-05-20 16:36:58,803 - Подключение от ('127.0.0.1', 52213)
2024-05-20 16:37:01,590 - Ошибка при обработке данных от клиента: 
2024-05-20 16:37:01,591 - Соединение с клиентом закрыто
2024-05-20 16:37:31,663 - Подключение от ('127.0.0.1', 52226)
2024-05-20 16:38:19,134 - Подключение от ('127.0.0.1', 52271)
2024-05-20 16:39:05,609 - Соединение с клиентом закрыто
2024-05-20 16:39:51,533 - Соединение с клиентом закрыто
2024-05-20 16:39:51,534 - Сервер закрыт
